In [46]:
import functions as fn
import pandas as pd
from importlib import reload
from tqdm import tqdm
from termcolor import colored
import matplotlib.pyplot as plt
from ooipy.tools import ooiplotlib as ooiplt
import datetime
from joblib import Parallel, delayed
import pandas as pd
import ooipy

In [47]:
import sys, os

# Disable
def blockPrint():
    sys.stdout = open(os.devnull, 'w')

# Restore
def enablePrint():
    sys.stdout = sys.__stdout__

In [48]:

df = pd.read_csv('E:/Acer backup 3/internships and jobs/UW/Reader grader and TA/Shima Abadi/Data/ais_2014B_2020B_V2.csv',sep = ',')


In [49]:
df['TIMESTAMP UTC']=df['TIMESTAMP UTC'].astype('datetime64[ns]')


In [50]:
df.columns

Index(['Unnamed: 0.1', 'Unnamed: 0', 'MMSI', 'SHIPNAME', 'VESSEL TYPE',
       'STATUS', 'SPEED (KNOTSx10)', 'LAT', 'LON', 'COURSE', 'HEADING',
       'TIMESTAMP UTC', 'LENGTH', 'Year', 'axial_Lat', 'axial_Lon',
       'axial_Loc', 'ship_Loc', 'central_caldera_Lat', 'central_caldera_Lon',
       'central_caldera_Loc', 'eastern_caldera_Lat', 'eastern_caldera_Lon',
       'eastern_caldera_Loc', 'distance(in km) axial',
       'distance(in km) central cald', 'distance(in km) eastern cald',
       'bearing_axial', 'bearing_eastern_cald', 'bearing_central_cald'],
      dtype='object')

In [51]:
df.shape

(2003830, 30)

In [52]:
df['TIMESTAMP UTC'].min()

Timestamp('2015-01-01 00:28:00')

In [53]:
df['TIMESTAMP UTC'].max()

Timestamp('2020-12-31 23:58:58')

### Remove airgun noise data

In [54]:
start_time=datetime.datetime(2019,7,1,0,0,0)
end_time=datetime.datetime(2019,8,1,0,0,0)


In [55]:
start_time

datetime.datetime(2019, 7, 1, 0, 0)

In [56]:
df=df[(df['TIMESTAMP UTC']<start_time) | (df['TIMESTAMP UTC'] > end_time)]
df.head()

,Unnamed: 0.1,Unnamed: 0,MMSI,SHIPNAME,VESSEL TYPE,STATUS,SPEED (KNOTSx10),LAT,LON,COURSE,...,central_caldera_Loc,eastern_caldera_Lat,eastern_caldera_Lon,eastern_caldera_Loc,distance(in km) axial,distance(in km) central cald,distance(in km) eastern cald,bearing_axial,bearing_eastern_cald,bearing_central_cald
0,0,0,209605000,AKILI,Bulk Carrier,0,108,46.09859,-129.6550,83,...,"(45.9549, -130.0089)",45.9399,-129.9742,"(45.9399, -129.9742)",32.25,31.72,30.37,193.760689,234.515164,239.809636
1,1,1,256832000,NaN,NaN,0,115,45.07486,-128.9430,178,...,"(45.9549, -130.0089)",45.9399,-129.9742,"(45.9399, -129.9742)",104.04,128.46,125.45,322.804182,320.490292,320.057435
2,2,2,352358000,ANNA G,Bulk Carrier,0,132,45.86138,-130.5627,86,...,"(45.9549, -130.0089)",45.9399,-129.9742,"(45.9399, -129.9742)",63.02,44.21,46.50,94.234035,78.935325,76.159593
3,3,3,356566000,GLOBAL SAIKAI,General Cargo,0,104,46.50261,-129.0129,272,...,"(45.9549, -130.0089)",45.9399,-129.9742,"(45.9399, -129.9742)",95.33,98.03,97.02,217.082385,230.114815,231.878577
4,4,4,477293500,JIN XIU FENG,Bulk Carrier,0,112,46.74633,-129.5912,299,...,"(45.9549, -130.0089)",45.9399,-129.9742,"(45.9399, -129.9742)",104.08,93.66,94.37,186.960616,198.291220,200.167826


In [57]:
min_duration=10
# path_to_write='E:/Acer backup 3/internships and jobs/UW/Reader grader and TA/Shima Abadi/Data/Benchmark_data/'
path_to_write='C:/Users/Khirod/UW/Shima Abadi - Ship Noise Data Set/Data with avg_time=1/Benchmark_Data/Ships/'


In [58]:
reload(fn)

<module 'functions' from 'C:\\Users\\khirod\\Documents\\GitHub\\Ocean-data-analysis\\functions.py'>

In [59]:
vessels_axial=pd.read_csv('Vessels/vessel_groups.csv')


In [60]:

vessels_axial.head()

,vessel,group3,ud_group,ud_group1,ud_group2,ud_group3
0,Bulk Carrier,Bulk,Cargo,Cargo typeC,Cargo,Cargo
1,Cargo,Dry Cargo,Cargo,Cargo typeB,Cargo,Cargo
2,Cargo/Containership,Dry Cargo,Cargo,Cargo typeC,Cargo,Cargo
3,Chemical Tanker,Wet Cargo,Tanker,Cargo typeA,Cargo,Tanker
4,Container Ship,Dry Cargo,Cargo,Cargo typeC,Cargo,Cargo


In [61]:
def get_benchmark_data(df,hydrophone_idx,inner_rad,outer_rad,min_duration,fmin=None,fmax=None,break_dur=True,get_acoustic=True,get_spectrogram=True):
    
    if hydrophone_idx==1:
        hydro='AB'
        hydro_file='Axial_Base'
    elif hydrophone_idx==2:
        hydro='CC'
        hydro_file='Central_Caldera'
    elif hydrophone_idx==3:
        hydro='EC'
        hydro_file='Eastern_Caldera'
    suffix=str(inner_rad)+'_'+ str(outer_rad)+'.csv'
    ais=fn.choose_df(df,hydrophone_idx)
    print(ais.head())
    isolated_ships=fn.get_isolated_ships(ais,vessels_axial,inner_rad,outer_rad,min_duration)
    isolated_ships.to_csv(path_to_write+hydro_file+'/'+hydro+'_isolated_ships_' + suffix)
    isolated_ais=fn.isolated_ais(ais,isolated_ships,vessels_axial,inner_rad)
    isolated_ais.to_csv(path_to_write+hydro_file+'/'+hydro+'_isolated_ais_'+ suffix)
    if break_dur:
        isolated_ais_10m=fn.break_duration(isolated_ships)
        isolated_ais_10m.to_csv(path_to_write+hydro_file+'/'+hydro+'_isolated_ais_10m_'+ suffix)
        
    spec_filename=path_to_write+hydro_file+'/Spectrograms'+'/'+hydro+'_'
    audio_filename=path_to_write+hydro_file+'/Audio'+'/'+hydro+'_'
    print(audio_filename)
    if get_acoustic==True:
        print('Saving wav files')
        for i in tqdm(range(len(isolated_ships))):
            
            min_time=isolated_ships.start_time[i]
            max_time=isolated_ships.end_time[i]
            str_mintime=min_time.strftime("%Y%m%d%H%M%S")
            str_maxtime=max_time.strftime("%Y%m%d%H%M%S")
            hydrophone_data=fn.get_acoustic(hydrophone_idx,min_time,max_time,fmin=fmin,fmax=fmax)
            if hydrophone_data==None:
                # print('data trace is none. Continuing to next')
                pass
            else:
                hydrophone_data.wav_write(filename=audio_filename+str_mintime+'_'+str_maxtime+'.wav')
    if get_spectrogram==True:
        print('Saving spectrograms')
        for i in tqdm(range(len(isolated_ships))):
            min_time=isolated_ships.start_time[i]
            max_time=isolated_ships.end_time[i]
            str_mintime=min_time.strftime("%Y%m%d%H%M%S")
            str_maxtime=max_time.strftime("%Y%m%d%H%M%S")
            spectrogram=fn.get_spectrogram_data(hydrophone_idx,min_time,max_time,fmin=fmin,fmax=fmax)
            if (spectrogram==None):
                # print('data trace is none. Continuing to next')
                pass
            else:
                if (spectrogram.values.shape[0]<2) | (spectrogram.values.shape[1]<2):
                    pass
                else:
                    #spectrogram.visualize(save_spec=True,plot_spec=False,filename=path_to_write+hydro+'/Spectrogram'+'/isolated_ships_'+str(inner_rad)+'_'+ str(outer_rad)+'_'+str(i)+'.png')
                    ooiplt.plot_spectrogram(spectrogram,plot=False,save=True,fmin=fmin,fmax=fmax,vmax=110,filename=spec_filename+str_mintime+'_'+str_maxtime+'.png')
    return isolated_ships,isolated_ais

In [62]:
df.columns

Index(['Unnamed: 0.1', 'Unnamed: 0', 'MMSI', 'SHIPNAME', 'VESSEL TYPE',
       'STATUS', 'SPEED (KNOTSx10)', 'LAT', 'LON', 'COURSE', 'HEADING',
       'TIMESTAMP UTC', 'LENGTH', 'Year', 'axial_Lat', 'axial_Lon',
       'axial_Loc', 'ship_Loc', 'central_caldera_Lat', 'central_caldera_Lon',
       'central_caldera_Loc', 'eastern_caldera_Lat', 'eastern_caldera_Lon',
       'eastern_caldera_Loc', 'distance(in km) axial',
       'distance(in km) central cald', 'distance(in km) eastern cald',
       'bearing_axial', 'bearing_eastern_cald', 'bearing_central_cald'],
      dtype='object')

In [63]:
def get_benchmark_data_parallel(df,hydrophone_idx,inner_rad,outer_rad,min_duration,fmin=0,fmax=100,break_dur=True,save_date=True):
    reload(fn)
    shared_link='https://uwnetid-my.sharepoint.com/:f:/r/personal/abadi_uw_edu/Documents/Ocean%20Data%20Lab%20SharePoint/OOI/Ship%20Noise%20Data%20Set/Data with avg_time=1/Benchmark_Data/'
    local_link='C:/Users/Khirod/UW/Shima Abadi - Ship Noise Data Set/Data with avg_time=1/Benchmark_Data/Ships/'
    if hydrophone_idx==1:
        hydro='AB'
        hydro_file='Axial_Base'
    elif hydrophone_idx==2:
        hydro='CC'
        hydro_file='Central_Caldera'
    elif hydrophone_idx==3:
        hydro='EC'
        hydro_file='Eastern_Caldera'
    suffix=str(inner_rad)+'_'+ str(outer_rad)+'.csv'
    ais=fn.choose_df(df,hydrophone_idx)
    isolated_ships=fn.get_isolated_ships(ais,vessels_axial,inner_rad,outer_rad,min_duration)
    isolated_ships.to_csv(path_to_write+hydro_file+'/'+hydro+'_isolated_ships_' + suffix)
   
    if break_dur:
        isolated_ais_10m=fn.break_duration(isolated_ships,hydro )
        isolated_ships=isolated_ais_10m.copy()
    
    isolated_ais=fn.isolated_ais(ais,isolated_ships,vessels_axial,inner_rad)
    isolated_ais.to_csv(path_to_write+hydro_file+'/'+hydro+'_isolated_ais_'+ suffix)
    print(isolated_ais_10m.head())
    
    ships_info1= isolated_ais.groupby(by='instance_id').agg({'SPEED (KNOTSx10)':['min','max','mean'],'LENGTH':['mean'],
                                                            'distance(in km)':['min','max','mean'],'COURSE':['min','max','mean'],
                                                            'HEADING':['min','max','mean']}).reset_index()
    ships_info1.columns=['instance_id','min_speed(KNOTSx10)','max_speed(KNOTSx10)','mean_speed(KNOTSx10)','length(in m)','min_distance(in kms)','max_distance(in kms)','mean_distance(in kms)',
                        'min_course','max_course','mean_course','min_heading','max_heading','mean_heading']
    
    ships_info2=isolated_ais[['instance_id','SHIPNAME']].drop_duplicates()
    
    
    isolated_ships=isolated_ships.merge(ships_info1,how='left',on='instance_id')
    isolated_ships=isolated_ships.merge(ships_info2,how='left',on='instance_id')
    

    
    
    
    if save_date==True:
        
        results = Parallel(n_jobs=8)(delayed(fn.save_spectrogram_acoustic_parallel)(hydrophone_idx,i,isolated_ships,inner_rad,outer_rad,path_to_write,fmin=fmin,fmax=fmax) for i in tqdm(range(len(isolated_ships))) )
    
    df_spec,df_audio,df_mseed,df_psds,df_psdpickle=fn.extend_ais(hydrophone_idx,shared_link,local_link)
    print(df_spec.head())
    isolated_ships=isolated_ships.merge(df_spec,how='left',on='instance_id')
    isolated_ships=isolated_ships.merge(df_audio,how='left',on='instance_id')
    isolated_ships=isolated_ships.merge(df_mseed,how='left',on='instance_id')
    isolated_ships=isolated_ships.merge(df_psds,how='left',on='instance_id')
    isolated_ships=isolated_ships.merge(df_psdpickle,how='left',on='instance_id')
    isolated_ships.to_csv(path_to_write+hydro_file+'/'+hydro+'_isolated_ais_10m_'+ suffix)
    

In [66]:
reload(fn)

<module 'functions' from 'C:\\Users\\khirod\\Documents\\GitHub\\Ocean-data-analysis\\functions.py'>

## Creating benchmark data for Axial Base

### Depth=2.6
### Axial base= 'AXABA1'



In [64]:
inner_rad= 5
outer_rad1=10
outer_rad2=20

In [168]:

d1,d2=get_benchmark_data(df,1,inner_rad,outer_rad1,min_duration,break_dur=True,get_acoustic=False,get_spectrogram=True)


 Max distance: 166.51 and Min distance: 0.01
        MMSI       SHIPNAME    VESSEL TYPE  SPEED (KNOTSx10)  COURSE  HEADING  \
0  209605000          AKILI   Bulk Carrier               108      83       84   
1  256832000            NaN            NaN               115     178      181   
2  352358000         ANNA G   Bulk Carrier               132      86       85   
3  356566000  GLOBAL SAIKAI  General Cargo               104     272      275   
4  477293500   JIN XIU FENG   Bulk Carrier               112     299      298   

        TIMESTAMP UTC  LENGTH  Year               ship_Loc       LAT  \
0 2015-01-01 00:28:00  189.99  2014   (46.09859, -129.655)  46.09859   
1 2015-01-01 00:28:00     NaN  2014   (45.07486, -128.943)  45.07486   
2 2015-01-01 00:28:00  229.00  2014  (45.86138, -130.5627)  45.86138   
3 2015-01-01 00:31:00  188.50  2014  (46.50261, -129.0129)  46.50261   
4 2015-01-01 00:48:00  229.00  2014  (46.74633, -129.5912)  46.74633   

        LON  distance(in km)       


100%|████████████████████████████████████████████████████████████████████████████████| 109/109 [00:02<00:00, 37.90it/s]


C:/Users/Khirod/UW/Shima Abadi - Ship Noise Data Set/Data/Benchmark data/Ships/Axial_Base/Audio/AB_audio_
Saving spectrograms



  0%|                                                                                          | 0/109 [00:00<?, ?it/s]

2016-11-25 22:49:56
2016-11-25 22:59:56
OO.AXBA1.AXBA1.HDH | 2016-11-25T22:49:56.000000Z - 2016-11-25T22:59:56.000000Z | 200.0 Hz, 120001 samples


...

2019-12-21 22:27:23
2019-12-21 22:37:23



 18%|██████████████▊                                                                  | 20/109 [01:01<04:58,  3.35s/it]

   Specific Time window timed out.
None
data trace is none. Continuing to next
2020-03-01 03:12:49
2020-03-01 03:22:49


 18%|██████████████▊                                                                  | 20/109 [01:04<04:45,  3.21s/it]


KeyboardInterrupt: 

In [79]:
print(d1['VESSEL TYPE'].value_counts(),'\n',d1['len_of_recording'].sum())

Research/Survey Vessel    47
Bulk Carrier              27
Vehicles Carrier           6
Special Vessel             4
General Cargo              4
Tug                        4
Oil Products Tanker        3
Passenger                  2
Oil/Chemical Tanker        2
Passenger Ship             2
Cargo/Containership        1
Towing Vessel              1
Crude Oil Tanker           1
Fishing Vessel             1
Wood Chips Carrier         1
Cargo                      1
LPG Tanker                 1
Yacht                      1
Name: VESSEL TYPE, dtype: int64 
 34860.48333333334


In [160]:

d1,d2=get_benchmark_data(df,1,inner_rad,outer_rad2,min_duration,break_dur=False,get_acoustic=False,get_spectrogram=False)

 Max distance: 166.51 and Min distance: 0.01


100%|████████████████████████████████████████████████████████████████████████████████| 132/132 [00:07<00:00, 18.14it/s]


In [2]:
print(d1['VESSEL TYPE'].value_counts(),'\n',d1['len_of_recording'].sum())

NameError: name 'd1' is not defined

In [133]:
# d1,d2=get_benchmark_data(df,1,5,30,min_duration,get_acoustic=False)

In [68]:
fixed_inner=5
fixed_outer=20
get_benchmark_data_parallel(df,1,fixed_inner,fixed_outer,min_duration,fmin=0,fmax=100,break_dur=True,save_date=True)

 Max distance: 166.51 and Min distance: 0.01



100%|██████████████████████████████████████████████████████████████████████████████| 3075/3075 [01:15<00:00, 40.88it/s]


        MMSI          start_time            end_time          VESSEL TYPE  \
0  210059000 2016-11-25 22:49:56 2016-11-25 22:59:56         Bulk Carrier   
2  212250000 2016-04-01 18:58:25 2016-04-01 19:08:25         Bulk Carrier   
3  212250000 2016-04-01 19:08:25 2016-04-01 19:18:25         Bulk Carrier   
5  212482000 2020-09-24 19:41:45 2020-09-24 19:51:45         Bulk Carrier   
7  220518000 2018-01-20 21:28:28 2018-01-20 21:38:28  Oil Products Tanker   

  ud_group  len_of_recording                       instance_id  
0    Cargo              10.0  AB_20161125224956_20161125225956  
2    Cargo              10.0  AB_20160401185825_20160401190825  
3    Cargo              10.0  AB_20160401190825_20160401191825  
5    Cargo              10.0  AB_20200924194145_20200924195145  
7   Tanker              10.0  AB_20180120212828_20180120213828  



100%|████████████████████████████████████████████████████████████████████████████| 3075/3075 [2:17:09<00:00,  2.68s/it]


                        instance_id  \
0  AB_20150319051501_20150319052501   
0  AB_20150420120408_20150420121408   
0  AB_20150420121408_20150420122408   
0  AB_20150420122408_20150420123408   
0  AB_20150420123408_20150420124408   

                                         spectrogram  
0  https://uwnetid-my.sharepoint.com/:f:/r/person...  
0  https://uwnetid-my.sharepoint.com/:f:/r/person...  
0  https://uwnetid-my.sharepoint.com/:f:/r/person...  
0  https://uwnetid-my.sharepoint.com/:f:/r/person...  
0  https://uwnetid-my.sharepoint.com/:f:/r/person...  


## Creating benchmark data for Central Caldera

### Depth=1.5
### Central Caldera= 'AXCC1'



In [140]:
inner_rad= 3
outer_rad1=6
outer_rad2=12


In [141]:
d1,d2=get_benchmark_data(df,2,inner_rad,outer_rad1,min_duration,break_dur=False,get_acoustic=False,get_spectrogram=False)

 Max distance: 168.72 and Min distance: 0.02


100%|████████████████████████████████████████████████████████████████████████████████| 105/105 [00:02<00:00, 40.09it/s]


In [142]:
print(d1['VESSEL TYPE'].value_counts(),'\n',d1['len_of_recording'].sum())

Research/Survey Vessel    67
Offshore Supply Ship      19
Bulk Carrier              10
Tug                        4
Special Vessel             3
Oil/Chemical Tanker        1
Tanker                     1
Name: VESSEL TYPE, dtype: int64 
 36766.350000000006


In [143]:
d1,d2=get_benchmark_data(df,2,inner_rad,outer_rad2,min_duration,break_dur=False,get_acoustic=False,get_spectrogram=False)

 Max distance: 168.72 and Min distance: 0.02


100%|████████████████████████████████████████████████████████████████████████████████| 111/111 [00:02<00:00, 42.45it/s]


In [144]:
print(d1['VESSEL TYPE'].value_counts(),'\n',d1['len_of_recording'].sum())

Research/Survey Vessel    73
Offshore Supply Ship      19
Bulk Carrier              10
Tug                        4
Special Vessel             3
Oil/Chemical Tanker        1
Tanker                     1
Name: VESSEL TYPE, dtype: int64 
 36359.26666666666


In [69]:
fixed_inner=5
fixed_outer=20
get_benchmark_data_parallel(df,2,fixed_inner,fixed_outer,min_duration,fmin=0,fmax=100,break_dur=True,save_date=True)

 Max distance: 168.72 and Min distance: 0.02
{'MMSI': 338415000, 'start_time': Timestamp('2019-08-30 18:32:35'), 'end_time': Timestamp('2019-08-30 18:42:35'), 'VESSEL TYPE': 'Offshore Supply Ship', 'ud_group': 'Offshore', 'len_of_recording': 10.0, 'instance_id': 'CC_20190830165625_20190830165845'}



100%|██████████████████████████████████████████████████████████████████████████████| 7271/7271 [02:59<00:00, 40.54it/s]


        MMSI          start_time            end_time   VESSEL TYPE ud_group  \
0  229343000 2017-02-05 14:38:01 2017-02-05 14:48:01  Bulk Carrier    Cargo   
2  229760000 2016-01-14 07:02:29 2016-01-14 07:12:29  Bulk Carrier    Cargo   
3  229760000 2016-01-14 07:12:29 2016-01-14 07:22:29  Bulk Carrier    Cargo   
5  232010034 2020-11-09 07:40:00 2020-11-09 07:50:00  Bulk Carrier    Cargo   
7  241124000 2015-10-29 16:28:17 2015-10-29 16:38:17  Bulk Carrier    Cargo   

   len_of_recording                       instance_id  
0              10.0  CC_20170205143801_20170205144801  
2              10.0  CC_20160114070229_20160114071229  
3              10.0  CC_20160114071229_20160114072229  
5              10.0  CC_20201109074000_20201109075000  
7              10.0  CC_20151029162817_20151029163817  



100%|████████████████████████████████████████████████████████████████████████████| 7271/7271 [5:09:27<00:00,  2.55s/it]


                        instance_id  \
0  CC_20150423074810_20150423075810   
0  CC_20150423075810_20150423080810   
0  CC_20150423080810_20150423081810   
0  CC_20150423081810_20150423082810   
0  CC_20150423082810_20150423083810   

                                         spectrogram  
0  https://uwnetid-my.sharepoint.com/:f:/r/person...  
0  https://uwnetid-my.sharepoint.com/:f:/r/person...  
0  https://uwnetid-my.sharepoint.com/:f:/r/person...  
0  https://uwnetid-my.sharepoint.com/:f:/r/person...  
0  https://uwnetid-my.sharepoint.com/:f:/r/person...  


In [42]:
fixed_inner=5
fixed_outer=20
get_benchmark_data_parallel(df,2,fixed_inner,fixed_outer,min_duration,fmin=0,fmax=100,break_dur=True,save_date=False)

 Max distance: 168.72 and Min distance: 0.02
{'MMSI': 338415000, 'start_time': Timestamp('2019-08-30 18:32:35'), 'end_time': Timestamp('2019-08-30 18:42:35'), 'VESSEL TYPE': 'Offshore Supply Ship', 'ud_group': 'Offshore', 'len_of_recording': 10.0, 'instance_id': 'CC_20190830165625_20190830165845'}


100%|██████████████████████████████████████████████████████████████████████████████| 7271/7271 [04:15<00:00, 28.44it/s]


        MMSI          start_time            end_time   VESSEL TYPE ud_group  \
0  229343000 2017-02-05 14:38:01 2017-02-05 14:48:01  Bulk Carrier    Cargo   
2  229760000 2016-01-14 07:02:29 2016-01-14 07:12:29  Bulk Carrier    Cargo   
3  229760000 2016-01-14 07:12:29 2016-01-14 07:22:29  Bulk Carrier    Cargo   
5  232010034 2020-11-09 07:40:00 2020-11-09 07:50:00  Bulk Carrier    Cargo   
7  241124000 2015-10-29 16:28:17 2015-10-29 16:38:17  Bulk Carrier    Cargo   

   len_of_recording                       instance_id  
0              10.0  CC_20170205143801_20170205144801  
2              10.0  CC_20160114070229_20160114071229  
3              10.0  CC_20160114071229_20160114072229  
5              10.0  CC_20201109074000_20201109075000  
7              10.0  CC_20151029162817_20151029163817  
                        instance_id  \
0  CC_20150423074810_20150423075810   
0  CC_20150423075810_20150423080810   
0  CC_20150423080810_20150423081810   
0  CC_20150423081810_201504230828

### In this particular example hydrophone is not properly collecting data for a particular duration

In [23]:
from datetime import datetime
min_time=datetime(2020,12,31,23,10,0)
max_time=datetime(2020,12,31,23,20,0)
x=df[(df['TIMESTAMP UTC']>=min_time) & (df['TIMESTAMP UTC']<=max_time)]

In [24]:
x

,Unnamed: 0.1,Unnamed: 0,MMSI,SHIPNAME,VESSEL TYPE,STATUS,SPEED (KNOTSx10),LAT,LON,COURSE,...,central_caldera_Loc,eastern_caldera_Lat,eastern_caldera_Lon,eastern_caldera_Loc,distance(in km) axial,distance(in km) central cald,distance(in km) eastern cald,bearing_axial,bearing_eastern_cald,bearing_central_cald
2003781,2003781,253382,351733000,CHESAPEAKE HIGHWAY,Vehicles Carrier,0,154,45.25223,-129.6331,28,...,"(45.9549, -130.0089)",45.9399,-129.9742,"(45.9399, -129.9742)",63.45,83.42,80.93,351.512094,340.981166,339.619824
2003782,2003782,253383,538002570,GENCO AUVERGNE,Bulk Carrier,0,72,46.83477,-129.1328,275,...,"(45.9549, -130.0089)",45.9399,-129.9742,"(45.9399, -129.9742)",122.85,118.77,118.67,203.075757,213.271337,214.796535
2003783,2003783,253384,477902900,GALAXY,Bulk Carrier,0,90,46.58752,-128.9267,232,...,"(45.9549, -130.0089)",45.9399,-129.9742,"(45.9399, -129.9742)",106.85,109.10,108.18,216.908858,228.573692,230.171670
2003784,2003784,253385,351733000,CHESAPEAKE HIGHWAY,Vehicles Carrier,0,157,45.26022,-129.6265,30,...,"(45.9549, -130.0089)",45.9399,-129.9742,"(45.9399, -129.9742)",62.66,82.77,80.26,350.929893,340.430626,339.075178
2003785,2003785,253386,538002570,GENCO AUVERGNE,Bulk Carrier,0,72,46.83505,-129.1391,276,...,"(45.9549, -130.0089)",45.9399,-129.9742,"(45.9399, -129.9742)",122.69,118.52,118.44,202.859564,213.064536,214.592982
2003786,2003786,253387,538002570,GENCO AUVERGNE,Bulk Carrier,0,67,46.83548,-129.1460,272,...,"(45.9549, -130.0089)",45.9399,-129.9742,"(45.9399, -129.9742)",122.53,118.26,118.19,202.619685,212.833498,214.365327
2003787,2003787,253388,351733000,CHESAPEAKE HIGHWAY,Vehicles Carrier,0,156,45.27167,-129.6167,31,...,"(45.9549, -130.0089)",45.9399,-129.9742,"(45.9399, -129.9742)",61.53,81.87,79.33,350.045079,339.607785,338.262600
2003788,2003788,253389,477902900,GALAXY,Bulk Carrier,0,90,46.58187,-128.9372,238,...,"(45.9549, -130.0089)",45.9399,-129.9742,"(45.9399, -129.9742)",105.86,108.08,107.16,216.758032,228.533977,230.147298
2003789,2003789,253390,351733000,CHESAPEAKE HIGHWAY,Vehicles Carrier,0,155,45.28058,-129.6090,32,...,"(45.9549, -130.0089)",45.9399,-129.9742,"(45.9399, -129.9742)",60.66,81.18,78.62,349.328593,338.950182,337.614107
2003790,2003790,253391,538002570,GENCO AUVERGNE,Bulk Carrier,0,70,46.83576,-129.1541,272,...,"(45.9549, -130.0089)",45.9399,-129.9742,"(45.9399, -129.9742)",122.32,117.94,117.88,202.341759,212.567615,214.103642


In [147]:
reload(fn)

spectrogram=fn.get_spectrogram_data(1,min_time,max_time,fmin=0,fmax=100)

OO.AXBA1.AXBA1.HDH | 2020-12-31T23:10:00.000000Z - 2020-12-31T23:20:00.000000Z | 200.0 Hz, 120001 samples


In [148]:
spectrogram.values.shape[0]

60

In [66]:
if (spectrogram.values.shape[0]<2) | (spectrogram.values.shape[1]<2):
    pass
else:
    ooiplt.plot_spectrogram(spectrogram,plot=False,save=True,fmin=0,fmax=100,filename='test.png')

## Creating benchmark data for Eastern Caldera

### Depth=1.5
### Eastern Caldera= 'AXEC2'


In [25]:
inner_rad= 3
outer_rad1=6
outer_rad2=12

In [149]:
d1,d2=get_benchmark_data(df,3,inner_rad,outer_rad1,min_duration,break_dur=False,get_acoustic=False,get_spectrogram=False)

 Max distance: 165.62 and Min distance: 0.01


100%|████████████████████████████████████████████████████████████████████████████████| 140/140 [00:03<00:00, 39.97it/s]


In [150]:
print(d1['VESSEL TYPE'].value_counts(),'\n',d1['len_of_recording'].sum())

Research/Survey Vessel    86
Offshore Supply Ship      23
Bulk Carrier              14
Tug                        7
Special Vessel             4
Fishing Vessel             2
Oil/Chemical Tanker        1
Oil Products Tanker        1
Chemical Tanker            1
Crude Oil Tanker           1
Name: VESSEL TYPE, dtype: int64 
 44975.23333333333


In [151]:
d1,d2=get_benchmark_data(df,3,inner_rad,outer_rad2,min_duration,break_dur=False,get_acoustic=False,get_spectrogram=False)

 Max distance: 165.62 and Min distance: 0.01


100%|████████████████████████████████████████████████████████████████████████████████| 149/149 [00:03<00:00, 43.53it/s]


In [153]:
print(d1['VESSEL TYPE'].value_counts(),'\n',d1['len_of_recording'].sum())

Research/Survey Vessel    95
Offshore Supply Ship      23
Bulk Carrier              14
Tug                        7
Special Vessel             4
Fishing Vessel             2
Oil/Chemical Tanker        1
Oil Products Tanker        1
Chemical Tanker            1
Crude Oil Tanker           1
Name: VESSEL TYPE, dtype: int64 
 44026.01666666666


In [ ]:
fixed_inner=5
fixed_outer=20
get_benchmark_data_parallel(df,3,fixed_inner,fixed_outer,min_duration,fmin=0,fmax=100,break_dur=True,save_date=True)

 Max distance: 165.62 and Min distance: 0.01



100%|██████████████████████████████████████████████████████████████████████████████| 7354/7354 [02:57<00:00, 41.54it/s]


        MMSI          start_time            end_time          VESSEL TYPE  \
0  247218600 2016-02-12 14:51:01 2016-02-12 15:01:01  Oil/Chemical Tanker   
1  247218600 2016-02-12 15:01:01 2016-02-12 15:11:01  Oil/Chemical Tanker   
2  247218600 2016-02-12 15:11:01 2016-02-12 15:21:01  Oil/Chemical Tanker   
3  247218600 2016-02-12 15:21:01 2016-02-12 15:31:01  Oil/Chemical Tanker   
4  247218600 2016-02-12 15:31:01 2016-02-12 15:41:01  Oil/Chemical Tanker   

  ud_group  len_of_recording                       instance_id  
0   Tanker              10.0  EC_20160212145101_20160212150101  
1   Tanker              10.0  EC_20160212150101_20160212151101  
2   Tanker              10.0  EC_20160212151101_20160212152101  
3   Tanker              10.0  EC_20160212152101_20160212153101  
4   Tanker              10.0  EC_20160212153101_20160212154101  



 84%|██████████████████████████████████████████████████████████████▎           | 6192/7354 [4:57:20<1:07:34,  3.49s/it]

In [44]:
fixed_inner=5
fixed_outer=20
get_benchmark_data_parallel(df,3,fixed_inner,fixed_outer,min_duration,fmin=0,fmax=100,break_dur=True,save_date=False)

 Max distance: 165.62 and Min distance: 0.01


100%|██████████████████████████████████████████████████████████████████████████████| 7354/7354 [03:55<00:00, 31.23it/s]


        MMSI          start_time            end_time          VESSEL TYPE  \
0  247218600 2016-02-12 14:51:01 2016-02-12 15:01:01  Oil/Chemical Tanker   
1  247218600 2016-02-12 15:01:01 2016-02-12 15:11:01  Oil/Chemical Tanker   
2  247218600 2016-02-12 15:11:01 2016-02-12 15:21:01  Oil/Chemical Tanker   
3  247218600 2016-02-12 15:21:01 2016-02-12 15:31:01  Oil/Chemical Tanker   
4  247218600 2016-02-12 15:31:01 2016-02-12 15:41:01  Oil/Chemical Tanker   

  ud_group  len_of_recording                       instance_id  
0   Tanker              10.0  EC_20160212145101_20160212150101  
1   Tanker              10.0  EC_20160212150101_20160212151101  
2   Tanker              10.0  EC_20160212151101_20160212152101  
3   Tanker              10.0  EC_20160212152101_20160212153101  
4   Tanker              10.0  EC_20160212153101_20160212154101  
                        instance_id  \
0  EC_20150116213100_20150116214100   
0  EC_20150116214100_20150116215100   
0  EC_20150117222100_201501172

## Comparison of various options

In [154]:
reload(fn)

def compare_hydrophone_records_axial(df,hydro_idx,inner_rad,outer_rad,min_duration,break_dur=True):
    print(colored('Comparative view with minimum duration: '+ str(min_duration),'green',attrs=['bold']))
    table=pd.DataFrame(columns=['hydrophone_idx','inner_rad','outer_rad','no_unique_ships','no_unique_vessels','no_instances'])
    for i_r in inner_rad:
        for o_r in outer_rad:
            print(o_r)
            if i_r<=o_r:
                temp_dict=dict()
                ais=fn.choose_df(df,hydro_idx,verbose=False)
                temp_df1=fn.get_isolated_ships(ais,i_r,o_r,min_duration)
                if break_dur:
                    temp_df2=fn.break_duration(temp_df1)
                    temp_dict['no_10m_durations']=len(temp_df2)
#                 print(len(temp_df1))
                temp_dict['hydrophone_idx']=hydro_idx
                temp_dict['inner_rad']=i_r
                temp_dict['outer_rad']=o_r
                temp_dict['no_unique_ships']=temp_df1['MMSI'].nunique()
                temp_dict['no_unique_vessels']=temp_df1['VESSEL TYPE'].nunique()
                temp_dict['no_instances']=len(temp_df1)
                temp_dict['len_of_recordings']=temp_df1.len_of_recording.sum()
                
                table=table.append(temp_dict,ignore_index=True)
    return table




In [155]:
inner_rad=[5]
outer_rad=[10,20]
min_duration=10
hydro_idx=1
table=compare_hydrophone_records_axial(df,hydro_idx,inner_rad,outer_rad,min_duration,break_dur=True)
table

Comparative view with minimum duration: 10
10
20


,hydrophone_idx,inner_rad,outer_rad,no_unique_ships,no_unique_vessels,no_instances,no_10m_durations,len_of_recordings
0,1.0,5.0,10.0,60.0,18.0,109.0,3543.0,34860.483333
1,1.0,5.0,20.0,53.0,17.0,132.0,3206.0,31362.316667


In [156]:
inner_rad=[3]
outer_rad=[6,12]
min_duration=10
hydro_idx=2
table=compare_hydrophone_records_axial(df,hydro_idx,inner_rad,outer_rad,min_duration,break_dur=True)
table

Comparative view with minimum duration: 10
6
12


,hydrophone_idx,inner_rad,outer_rad,no_unique_ships,no_unique_vessels,no_instances,no_10m_durations,len_of_recordings
0,2.0,3.0,6.0,26.0,7.0,105.0,3735.0,36766.350000
1,2.0,3.0,12.0,26.0,7.0,111.0,3699.0,36359.266667


In [157]:
inner_rad=[3]
outer_rad=[6,12]
min_duration=10
hydro_idx=3
table=compare_hydrophone_records_axial(df,hydro_idx,inner_rad,outer_rad,min_duration,break_dur=True)
table

Comparative view with minimum duration: 10
6
12


,hydrophone_idx,inner_rad,outer_rad,no_unique_ships,no_unique_vessels,no_instances,no_10m_durations,len_of_recordings
0,3.0,3.0,6.0,32.0,10.0,140.0,4571.0,44975.233333
1,3.0,3.0,12.0,32.0,10.0,149.0,4482.0,44026.016667


In [110]:
# reload(fn)

# print(colored('Overlapping of all hydrophones with minimum number of timestamps: '+ str(min_duration),'green',attrs=['bold']))
# table2=pd.DataFrame(columns=['inner_rad','outer_rad','no_unique_ships','no_unique_vessels','no_records'])

# for i_r in inner_rad:
#     for o_r in outer_rad:
#         combined_df=pd.DataFrame()
#         for i in range(1,4,1):
#             temp_dict=dict()
#             ais=fn.choose_df(df,i,verbose=False)
#             d1=fn.get_isolated_ships(ais,i_r,o_r,min_duration)
#             combined_df=pd.concat([combined_df,d1])

#         temp_dict['inner_rad']=i_r
#         temp_dict['outer_rad']=o_r
#         temp_dict['no_unique_ships']=combined_df['MMSI'].nunique()
#         temp_dict['no_unique_vessels']=combined_df['VESSEL TYPE'].nunique()
#         temp_dict['no_records']=len(combined_df)
#         table2=table2.append(temp_dict,ignore_index=True)
# table2
        